In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd

import requests
from pprint import pprint
import folium




In [2]:
app = dash.Dash(__name__)

def get_lat_lon(streetname):
    BASE_URL = 'https://nominatim.openstreetmap.org/search?format=json'
    response = requests.get(f'{BASE_URL}&street={streetname}&city=Frankfurt')
    data = response.json()
    
    if data:
        lat = data[0].get('lat')
        lon = data[0].get('lon')
        return float(lat), float(lon)
    else:
        # Handle the case where the geocoding service does not return valid data
        return None

In [3]:
# Layout of the app
app.layout = html.Div([
    html.H1("Bike-Friendly Route Planner"),
    
    # Input for start location
    dcc.Input(
        id='start-location',
        type='text',
        placeholder='Enter your start location...',
        value='',
    ),
    
    # Input for destination
    dcc.Input(
        id='destination',
        type='text',
        placeholder='Enter your destination...',
        value='',
    ),
    
    # Dropdown for route type
    dcc.Dropdown(
        id='route-type',
        options=[
            {'label': 'Fastest Route', 'value': 'fastest'},
            {'label': 'Bike-Friendly Route', 'value': 'bike_friendly'},
            # Add more options as needed
        ],
        value='fastest',  # Default value
    ),
    
    # Button to trigger route calculation
    html.Button('Find Route', id='find-route-button'),
    
    # Map to display the route
    dcc.Graph(id='route-map'),
])

In [4]:
# Callback to update the map based on user input
@app.callback(
    Output("route-map", "figure"),
    [Input("find-route-button", "n_clicks"),],
    [State("start-location", "value"),
     State("destination", "value"),]
)
def update_map(n_clicks, start_location, destination):
    if n_clicks is None:
        return folium.Map(location=[50.110924, 8.682127], width=1200, height=720)
    
    # Your existing code to get latitude and longitude for start and destination
    start_data = get_lat_lon(start_location)
    dest_data = get_lat_lon(destination)

    start_coords = float(start_data[0]), float(start_data[1])
    dest_coords = float(dest_data[0]), float(dest_data[1])

    # Create folium map
    m = folium.Map(location=start_coords, width=1200, height=720)

    # Add markers and polyline
    folium.Marker(start_coords, popup=f'Start: {start_location}').add_to(m)
    folium.Marker(dest_coords, popup=f'Destination: {destination}').add_to(m)
    folium.PolyLine([start_coords, dest_coords], color="blue").add_to(m)

    # Convert folium map to Plotly figure
    fig = folium_static.to_figure(m)

    return fig

In [5]:
if __name__ == '__main__':
    app.run_server(debug=True)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File c:\Users\Zhenzhen\anaconda3\Lib\site-packages\dash\_callback.py:483, in register_callback.<locals>.wrap_func.<locals>.add_context(*args=(None, '', ''), **kwargs={})
    482 try:
--> 483     jsonResponse = to_json(response)
        response = {'multi': True, 'response': defaultdict(<class 'dict'>, {'route-map': {'figure': <folium.folium.Map object at 0x000001858B9A2E50>}})}
    484 except TypeError:

File c:\Users\Zhenzhen\anaconda3\Lib\site-packages\dash\_utils.py:26, in to_json(
    value={'multi': True, 'response': defaultdict(<class 'dict'>, {'route-map': {'figu...olium.folium.Map object at 0x000001858B9A2E50>}})}
)
     24 from plotly.io.json import to_json_plotly
---> 26 return to_json_plotly(value)
        value = {'multi': True, 'response': defaultdict(<class 'dict'>, {'route-map': {'figure': <folium.folium.Map object at 0x0